# Moving Average Algo V1.0

In [ ]:
""" Please note that this is a work in progress.
It uses ib_insync for execution and yfinance for Market Data.
This is just for Daily Analysis at the moment before applying this to strategy to anything quicker than a daily strategy,
I suggest using a fee service for reliability."""

### Import Libraries
import math
import pandas as pd
import numpy as np
import requests as r
import yfinance as yf
from datetime import datetime
datetime.now()

In [1]:
### must be connected to interactive brokers
from ib_insync import *
util.startLoop() #Use when using jupyter notebooks
ib = IB()
ib.connect('127.0.0.1', 4001, clientId=17) # ibgateway
ib.sleep(2)

Peer closed connection. clientId 17 already in use?
API connection failed: TimeoutError()


TimeoutError: 

In [ ]:
### test Inteactive Brokers Connectivity
ib.isConnected()

In [ ]:
# check current positions
ib.positions()

In [ ]:
# Below call will list all available dimensions via yfinance
# yf.Ticker(etf).info

In [ ]:
etf='USO'
etf_52l=yf.Ticker(etf).info['fiftyTwoWeekLow']
etf_52h=yf.Ticker(etf).info['fiftyTwoWeekHigh']
etf_bid=yf.Ticker(etf).info['bid']
etf_ask=yf.Ticker(etf).info['ask']
etf_last=((etf_bid+etf_ask)/2) #used to set up trade conditions in the premarket
etf_52l, etf_52h, etf_last

In [ ]:
# convert etf_last to a dataframe concat
last=pd.DataFrame({'Close': etf_last}, index=['Today'])
last

In [ ]:
# download historical data, for strategy, please note that you can change history to shorter intervals but please note that
# yfinance does give 'noisy' data!
etf_hist = yf.download(etf, start=datetime(2019,1,1), end=datetime(2024,6,26))['Close']

In [ ]:
etf_hist=pd.DataFrame(etf_hist)
# etf_hist=pd.concat([etf_hist, last]) use in premarket
etf_hist['sma5']=etf_hist.Close.rolling(5).mean()
etf_hist['sma10']=etf_hist.Close.rolling(10).mean()
etf_hist['sma20']=etf_hist.Close.rolling(20).mean()
# etf=etf.rename({'Close': 'close'}, axis='columns')
etf_hist.dropna(inplace=True)
etf_hist=etf_hist.round(2)
display(etf_hist['sma5'].iloc[-1], etf_hist['sma10'].iloc[-1],etf_hist['sma20'].iloc[-1])
etf_hist.tail()

In [ ]:
# retrieve balance
#to see all available tags, run ib.accountValues()
balance = float([x.value for x in ib.accountValues() if x.tag=='AvailableFunds' and x.currency=='USD'][0])
# calculate risk and share purchase amount
max_risk = math.floor(((balance*.02)/((etf_hist['sma10'].iloc[-1]+.03)-(etf_hist['sma20'].iloc[-1]))))
optimal_cost = ((etf_hist['sma10'].iloc[-1]+.03) * max_risk).round(2)
# if you would like to set your share amount and see your cost
size=250
cost = (etf_hist['sma10'].iloc[-1]+.03) * size
#snapshot:
balance,max_risk, optimal_cost, cost 

In [ ]:
# set contract and size for interactive brokers
etf1 = Stock(symbol = etf, exchange = 'SMART', currency = 'USD')
ib.qualifyContracts(etf1)

In [ ]:
# if sma5>sma10>sma20 then buy etf @ sma10
long_order = LimitOrder(action='BUY', totalQuantity=max_risk, lmtPrice=etf_hist['sma10'].iloc[-1], outsideRth=True)
long_priceCondition = PriceCondition(conId=etf1.conId, exch='SMART', isMore=False, price=etf_hist['sma10'].iloc[-1])
long_order.conditions.append(long_priceCondition)
if etf_hist['sma5'].iloc[-1]>etf_hist['sma10'].iloc[-1]>etf_hist['sma20'].iloc[-1]:
    trade_long=ib.placeOrder(etf1,long_order)

In [ ]:
# check order status
trade_long.orderStatus

In [ ]:
# check open orders
ib.openOrders()

In [ ]:
# cancel all open orders
for order in ib.openOrders():
    ib.cancelOrder(order)

In [ ]:
# close long orders, if an order is filled, this will have to be ran.
#close on profit
co_tp=LimitOrder(action='SELL', totalQuantity=max_risk, lmtPrice=etf_52h -.25, outsideRth=True)
co_tp_priceCondition = PriceCondition(conId=etf1.conId, exch='SMART', isMore=True, price=etf_52h -.25)
co_tp.conditions.append(co_tp_priceCondition)
#close on lost
co_sl=LimitOrder(action='SELL', totalQuantity=max_risk, lmtPrice=etf_hist['sma20'].iloc[-1] - .25, outsideRth=True)
co_sl_priceCondition = PriceCondition(conId=etf1.conId, exch='SMART', isMore=False, price=etf_hist['sma20'].iloc[-1] - .24)
co_sl.conditions.append(co_sl_priceCondition)
if len(ib.positions()) > 0:
    ct_tp=ib.placeOrder(etf1,co_tp)
    ct_sl=ib.placeOrder(etf1,co_sl)


In [2]:
# current positions
ib.positions()

[]

In [ ]:
# trade summary 
ib.trades()